## Portfolio Exercise: Starbucks
<br>

<img src="https://opj.ca/wp-content/uploads/2018/02/New-Starbucks-Logo-1200x969.jpg" width="200" height="200">
<br>
<br>
 
#### Background Information

The dataset you will be provided in this portfolio exercise was originally used as a take-home assignment provided by Starbucks for their job candidates. The data for this exercise consists of about 120,000 data points split in a 2:1 ratio among training and test files. In the experiment simulated by the data, an advertising promotion was tested to see if it would bring more customers to purchase a specific product priced at $10. Since it costs the company 0.15 to send out each promotion, it would be best to limit that promotion only to those that are most receptive to the promotion. Each data point includes one column indicating whether or not an individual was sent a promotion for the product, and one column indicating whether or not that individual eventually purchased that product. Each individual also has seven additional features associated with them, which are provided abstractly as V1-V7.

#### Optimization Strategy

Your task is to use the training data to understand what patterns in V1-V7 to indicate that a promotion should be provided to a user. Specifically, your goal is to maximize the following metrics:

* **Incremental Response Rate (IRR)** 

IRR depicts how many more customers purchased the product with the promotion, as compared to if they didn't receive the promotion. Mathematically, it's the ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group (_treatment_) minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group (_control_).

$$ IRR = \frac{purch_{treat}}{cust_{treat}} - \frac{purch_{ctrl}}{cust_{ctrl}} $$


* **Net Incremental Revenue (NIR)**

NIR depicts how much is made (or lost) by sending out the promotion. Mathematically, this is 10 times the total number of purchasers that received the promotion minus 0.15 times the number of promotions sent out, minus 10 times the number of purchasers who were not given the promotion.

$$ NIR = (10\cdot purch_{treat} - 0.15 \cdot cust_{treat}) - 10 \cdot purch_{ctrl}$$

For a full description of what Starbucks provides to candidates see the [instructions available here](https://drive.google.com/open?id=18klca9Sef1Rs6q8DW4l7o349r8B70qXM).

Below you can find the training data provided.  Explore the data and different optimization strategies.

#### How To Test Your Strategy?

When you feel like you have an optimization strategy, complete the `promotion_strategy` function to pass to the `test_results` function.  
From past data, we know there are four possible outomes:

Table of actual promotion vs. predicted promotion customers:  

<table>
<tr><th></th><th colspan = '2'>Actual</th></tr>
<tr><th>Predicted</th><th>Yes</th><th>No</th></tr>
<tr><th>Yes</th><td>I</td><td>II</td></tr>
<tr><th>No</th><td>III</td><td>IV</td></tr>
</table>

The metrics are only being compared for the individuals we predict should obtain the promotion – that is, quadrants I and II.  Since the first set of individuals that receive the promotion (in the training set) receive it randomly, we can expect that quadrants I and II will have approximately equivalent participants.  

Comparing quadrant I to II then gives an idea of how well your promotion strategy will work in the future. 

Get started by reading in the data below.  See how each variable or combination of variables along with a promotion influences the chance of purchasing.  When you feel like you have a strategy for who should receive a promotion, test your strategy against the test dataset used in the final `test_results` function.

In [1]:
# load in packages
from itertools import combinations

from test_results import test_results, score
import numpy as np
import pandas as pd
import scipy as sp
import sklearn as sk

import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

# load in the data
train_data = pd.read_csv('./training.csv')
train_data.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2


In [ ]:
train_data.isna().sum(axis=0)

In [ ]:
train_data.drop(columns=["ID"], inplace=True)

In [ ]:
train_data["Promotion"] = train_data["Promotion"].map(lambda x: int(x == "Yes"))

In [ ]:
train_data.rename(columns={"Promotion": "promotion"}, inplace=True)

# Examine each variable

In [ ]:
train_data.describe().round(2)

In [ ]:
cols = ["V1", "V2", "V3", "V4", "V5", "V6", "V7"]

In [ ]:
stats_00 = train_data.loc[(train_data["promotion"]==0) & (train_data["purchase"]==0), cols].mean()
stats_01 = train_data.loc[(train_data["promotion"]==0) & (train_data["purchase"]==1), cols].mean()
stats_10 = train_data.loc[(train_data["promotion"]==1) & (train_data["purchase"]==0), cols].mean()
stats_11 = train_data.loc[(train_data["promotion"]==1) & (train_data["purchase"]==1), cols].mean()

These are the statistics for users that 

In [ ]:
print("Average features for users that did not receive promo and did not purchase: \n{}\n".format(stats_00))
print("Average features for users that did not receive promo and did purchase: \n{}\n".format(stats_01))
print("Difference: \n{}".format(stats_00-stats_01))

In [ ]:
print("Average features for users that did receive promo and did not purchase: \n{}\n".format(stats_10))
print("Average features for users that did receive promo and did purchase: \n{}\n".format(stats_11))
print("Difference: \n{}".format(stats_10-stats_11))

V3 seems quite important.

# Descriptive statistical analysis between groups

In [ ]:
train_data.groupby("purchase").mean()

In [ ]:
lm = LinearRegression(normalize=True)

In [4]:
X = train_data.iloc[:, [0, 2, 3, 4, 5, 6, 7, 8]]
y = train_data.iloc[:, 1]

In [ ]:
lm.fit(X, y)

In [ ]:
train_data[train_data["promotion"]==1].groupby("purchase").mean()

In [ ]:
lm.coef_

In [2]:
def promotion_strategy(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''
    promotion = pd.DataFrame({"Promotion": ["Yes" for _ in range(df.shape[0])]})
    
    return promotion

In [5]:
promotion_strategy(X)

,Promotion
0,Yes
1,Yes
2,Yes
3,Yes
4,Yes
5,Yes
6,Yes
7,Yes
8,Yes
9,Yes


In [ ]:
# This will test your results, and provide you back some information 
# on how well your promotion_strategy will work in practice

test_results(promotion_strategy)